In [39]:
import torch

In [45]:
a = torch.ones(2, 2) # 缺失情况下默认 requires_grad = False
a += 1
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
print(a.grad_fn) # 虽然有计算，但无其他的tensor参与，所以还是None
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum() #求和
print(b)
print(b.grad_fn) # 最后执行的求和操作

False
None
True
tensor(144., grad_fn=<SumBackward0>)


In [40]:
x = torch.ones(2,2, requires_grad=True)
print(x)
print(x.grad_fn) #x是直接生成的，不是由其他的tensor生成而来

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [41]:
y = x + 2
print(y)
print(y.grad_fn)#y是由x所组成的，并且最后执行的是加法

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [42]:
print(x.is_leaf, y.is_leaf) # True False  .is_leaf()判断是否为叶子节点

True False


In [43]:
z = y * y * 3
out = z.mean()#求均值
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [44]:
out.backward()#计算梯度，反向传播到组成out的变量

In [50]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [54]:
x.grad.data.zero_()
out2 = x.sum()
out2.backward()
print(x.grad)
out3 = x.sum()
#x.grad.data.zero_()
out3.backward()
print(x.grad)#grad在反向传播过程中是累加的(accumulated)，这意味着每⼀次运⾏反向传播，梯度都会累加之前的梯度，所以⼀般在反向传播之前需把梯度清零。

tensor([[1., 1.],
        [1., 1.]])
tensor([[2., 2.],
        [2., 2.]])


In [55]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True) #不允许张量对张量求导，只允许标量对张量求导，求导结果是和⾃变量同形的张量
y = 2 * x
z = y.view(2, 2)
v = torch.tensor([[1,2],[3,4]]) # 
z.backward(v) #传入同型矩阵将张量z进行加权求和，将z变为标量
print(x.grad)

tensor([2., 4., 6., 8.])


In [59]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
 y2 = x ** 3
y3 = y1 + y2
 
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y2.grad_fn) #中断追踪时，连Function都取消关联了
print(y3, y3.requires_grad) # True

y3.backward()
print(x.grad) #其中y3关于y2，dy3/dy2 * dy2/dx的部分不会被计算

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
None
tensor(2., grad_fn=<AddBackward0>) True
tensor(2.)


In [76]:
x = torch.ones(1,requires_grad=True)
print(x.data) # 还是⼀个tensor
print(x.data.requires_grad) # 但是已经是独⽴于计算图之外

#x *= 100  # 报错！尝试原地修改叶子张量
#x = x * 100  # 合法：创建新张量（非原地操作）
y = 2 * x
#x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([1.], requires_grad=True)
tensor([2.])
